# このノートブックをベースにデータ分析基盤を作成していきます

docker exec -it pyspark_big /bin/bash

pyspark-topic

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka_big:9092 
```

# データソース(Postgres)からデータを取得してみましょう
今回は、コンテナ名「metabase_postgres_big」のmetabaseデータベースからordersテーブルのデータをembulkを使って取得してみましょう。

>>> df = spark.read.parquet("/tmp/share_file/datalake/")
>>> df.show()  


テーブルの作成はいらない
psql -h postgres_data_big -p 5433 -U postgres -d access_layer

```
create database access_layer
create table if not exists etl_load (
       product_id           varchar(255)             default null,
       user_id           varchar(255)             default null
);

```

pyspark --packages org.apache.spark:spark-streaming_2.13:3.2.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2,org.apache.spark:spark-avro_2.12:3.2.2

spark-submit --packages org.apache.spark:spark-streaming_2.13:3.2.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2,org.apache.spark:spark-avro_2.12:3.2.2 

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka_big:9092") \
  .option("subscribe", "pyspark-topic") \
  .load()

# メモリシンク
memory_sink = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("web_actions") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/web_memory/") \
  .start()